# monoid

## newtype

In [3]:
[(+1), (*100), (*5)] <*> [1,2,3]
-- 리스트를 applicative functor 로 만드는 방법
-- 1) list comprehension 연산 만들기

[2,3,4,100,200,300,5,10,15]

In [7]:
import qualified Control.Applicative as Applicative

Applicative.getZipList $ Applicative.ZipList [(+1), (*100), (*5)] <*> Applicative.ZipList [1,2,3]

-- 리스트를 applicative functor 로 만드는 방법
-- 2) 왼쪽 함수를 오른쪽 펑터 값에 적용하기

[2,200,15]

In [18]:
data ZipList a = ZipList [a]

Line 1: Use newtype instead of data
Found:
data ZipList a = ZipList [a]
Why not:
newtype ZipList a = ZipList [a]

In [13]:
data Nomad a = Nomad a 
    deriving (Show)
Nomad 4

Line 1: Use newtype instead of data
Found:
data Nomad a
  = Nomad a
  deriving Show
Why not:
newtype Nomad a
  = Nomad a
  deriving Show

Nomad 4

In [19]:
data ZipList a = ZipList { getZipList :: [a] }

Line 1: Use newtype instead of data
Found:
data ZipList a = ZipList {getZipList :: [a]}
Why not:
newtype ZipList a = ZipList {getZipList :: [a]}

In [21]:
newtype ZipList a = ZipList {getZipList :: [a]}
-- data를 사용하면, 프로그램 중에 래핑/언래핑 비용이 발생하지만,
-- newtype을 사용하면, 내부적으로는 동일하지만 다른 타입을 만듬
-- newtype 사용시에 하스켈은 기존 타입을 래핑하기 위해서 사용한다는 것을 알고, 래핑 언래핑 작업을 제거한다.

In [26]:
-- newtype 키워드를 통해 기존 타입을 새로운 타입으로 만들면
-- 1) 단하나의 값 생성자만 만들수 있으며
-- 2) 값 생성자는 단 하나의 필드만 가질 수 있다.

newtype TestNewType = TestNewType {callOfDuty :: String}

a = TestNewType "army"
:t a
b = callOfDuty a
b

a :: TestNewType

"army"

In [27]:
newtype CharList = CharList {getCharList :: [Char] } deriving (Eq, Show)

In [30]:
CharList "this will be shown!"
CharList "benny" == CharList "benny"
CharList "benny" == CharList "oisters"

CharList {getCharList = "this will be shown!"}

True

False

In [32]:
:t CharList
:t getCharList

CharList :: [Char] -> CharList

getCharList :: CharList -> [Char]

## 타입 클래스 인스턴스를 만들기 위해 newtype 사용하기

In [33]:
newtype Pair b a = Pair { getPair :: (a,b) }

In [36]:
instance Functor (Pair c) where
    fmap f (Pair (x, y)) = Pair (f x , y)
    -- fmap :: (a -> b) -> f a -> f b
    -- fmap :: (a, b) -> (a, c) -> f (a, c) -> f (b, c)
    -- f (_, c) => Pair c

In [38]:
getPair $ fmap (*100)  (Pair (2,3))
getPair $ fmap reverse (Pair ("london calling", 3))

(200,3)

("gnillac nodnol",3)

In [39]:
--하스켈의 undefined는 잘못된 연산을 나타낸다.
--undefined = error "Prelude.undefined"
undefined

: 

In [40]:
head [3,4,5, undefined, 2, undefined]

3

In [43]:
data CoolBool = CoolBool { getCoolBool :: Bool }
helloMe :: CoolBool -> String
helloMe (CoolBool _) = "hello"

helloMe undefined

-- 예외가 발생한 이유
-- data 키워드로 정의된 타입은 여러개의 값 생성자들을 가질수 있음
-- 함수는 주어진 값이 CoolBool 패턴에 맞는지 알아보려면 값을 만들때 사용한 값 생성자가 무엇인지
-- 그 값을 확인해야한다
-- 그리고 undefined를 판단하려고 할 때 예외가 발생한다.

Line 1: Use newtype instead of data
Found:
data CoolBool = CoolBool {getCoolBool :: Bool}
Why not:
newtype CoolBool = CoolBool {getCoolBool :: Bool}

: 

In [46]:
newtype CoolBool = CoolBool { getCoolBool :: Bool }
helloMe :: CoolBool -> String
helloMe (CoolBool _) = "hello"

helloMe undefined
-- 하스켈에서 newtype의 패턴매칭이 더 느긋하며, data 보다 처리속도는 더 빠르다

-- newtype은 원래 값을 동일한 방법으로 새운 타입의 값들을 나타낼 수 있음
-- newtype은 하나의 생성자만 가지므로, 값 생성자를 체크하기 위해 값을 판단할 필요가 없기 때문

"hello"

In [48]:
-- 이것은 data와 newtype 이 비슷해보일지 몰라도, 다른 매커니즘이라는 것
-- data는 타입을 처음부터 만드는데 사용할 수 있다
-- newtype은 기존 타입으로 전혀 새로운 타입을 만드는데 사용된다.
-- newtype 값의 패턴매칭은, data에서처럼 상자로 어떤 값을 갖는다기 보다는, 
-- (박싱이 아니라 캐스팅?)
-- 하나의 타입에서 다른 타입으로 직접 변환하는 것이다.

In [49]:
newtype CharList = CharList {getCharList :: [Char]}

-- CharList와 [Char]을 합치기 위해 ++ 를 사용할 수 없다.
-- CharList끼리 합치기 위해서도 ++를 사용할 수 없다.
-- CharList가 [Char] 리스트를 포함하고 있다고 말할 수는 있어도 CharList 타입이 [Char]가 아니기 때문

In [50]:
-- newtype 안에 레코드 구문을 사용하면 새로운 타입과 원래의 타입을 서로 변환해주느는 함수들을 얻는다
-- 새로운 타입은 원래 타입이 속한 타입 클래스와 인스턴스를 자동으로 만들어주지 않는다.

## 모노이드

In [54]:
:info Monoid
-- type Monoid :: * -> Constraint
-- class Semigroup a => Monoid a where
--   mempty :: a
--   mappend :: a -> a -> a
--   mconcat :: [a] -> a

type Monoid :: * -> Constraint
class Semigroup a => Monoid a where
  mempty :: a
  mappend :: a -> a -> a
  mconcat :: [a] -> a
  {-# MINIMAL mempty #-}
  	-- Defined in ‘GHC.Base’
instance forall a k (b :: k). Monoid a => Monoid (Applicative.Const a b) -- Defined in ‘Data.Functor.Const’
instance Monoid Display -- Defined in ‘IHaskell.Types’
instance Monoid [a] -- Defined in ‘GHC.Base’
instance Monoid Ordering -- Defined in ‘GHC.Base’
instance Semigroup a => Monoid (Maybe a) -- Defined in ‘GHC.Base’
instance Monoid a => Monoid (IO a) -- Defined in ‘GHC.Base’
instance Monoid b => Monoid (a -> b) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b, Monoid c, Monoid d, Monoid e) => Monoid (a, b, c, d, e) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b, Monoid c, Monoid d) => Monoid (a, b, c, d) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b, Monoid c) => Monoid (a, b, c) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b) => Monoid (a, b) -- Defined in ‘GHC.Base’
instance Monoid () -- Defined in ‘GHC.Base’
instance Monoid a => Monoid (IHaskellTH.Q a) -- Defined in ‘Language.Haskell.TH.Instances’

In [58]:
-- 모노이드는 구체적인 타입(concrete type)만을 내부 값으로 가진다
-- Functor나 Applicative와는 다른 점

In [ ]:
-- 모노이드는 연결 이진함수와 항등원으로 구성된다
-- 어떤것이 함수에 대한 식별자(항등원)처럼 동작한다면 

In [61]:
import qualified Data.Monoid as Monoid

:t Monoid.mempty
-- 매개변수를 받지 않는다
-- Bounded의 minBound 같은 다형성 상수
-- 특정 모노이드에 대한 식별값을 나타냄

Monoid.mempty :: forall a. Monoid a => a

In [63]:
import qualified Data.Monoid as Monoid

:t Monoid.mappend
-- 동일한 타입 두개를 받아 또 다른 값을 반환
-- 어떤 방법으로 두 가지를 이용하여 연산하는 것을 의미

Monoid.mappend :: forall a. Monoid a => a -> a -> a

In [64]:
import qualified Data.Monoid as Monoid

:t Monoid.mconcat
-- 모노이드 값의 리스트를 받아서, 리스트 요소간에 mappend를 적용하여 하나의 값으로 만든다 
-- 리스트를 오른쪽에서부터 fold한다.
-- 디폴트 구현체가 있으므로 별도로 구현할 필요가 없다

Monoid.mconcat :: forall a. Monoid a => [a] -> a

## 모노이드 규칙

1. mempty `mappend` x = x
2. x `mappend` mempty = x
3. (x `mappend` y) `mappend` z = x `mappend` (y `mappend` z)

1,2는 mempty가 항등원으로 역할해야함을 의미하며
3은 mappend가 결합성(associativity, 결합 순서에 연관없이 값이 같음)이 있어야함을 의미함

## 리스트는 모노이드

In [ ]:
instance Monoid [a] where  
    -- [a]는 concrete type 이다
    mempty = []  
    mappend = (++)  

In [73]:
[1,2,3] `mappend` [4,5,6]
("one" `mappend` "two") `mappend` "tree"
"one" `mappend` ("two" `mappend` "tree")
"one" `mappend` "two" `mappend` "tree"
"pang" `mappend` mempty
mconcat [[1,2], [3,6], [9]]
mempty :: [a] 
-- ghci에서 됨, 
-- 빈리스트는 모든 타입을 담고 있는 것처럼 행동할 수 있기 때문에 [a]와 같은 일반적인 타입을 사용할 수 있다

Line 5: Monoid law, right identity
Found:
"pang" `mappend` mempty
Why not:
"pang"

[1,2,3,4,5,6]

"onetwotree"

"onetwotree"

"onetwotree"

"pang"

[1,2,3,6,9]

: 

In [75]:
"one" `mappend` "two"
"two" `mappend` "one"

-- 모노이드는 순서가 달라도 값이 같아야함을 규정하지는 않는다.

"onetwo"

"twoone"

## 곱셈과 덧셈도 모노이드

In [76]:
0 + 4
5 + 0
(1 + 3) + 5
1 + (3 + 5)

4

5

9

9

In [ ]:
import qualified Data.Monoid as Monoid
-- 하나의 타입이 여러 같은 타입 인스턴스가 될수는 없으므로 newtype Product와 Sum이 별도로 존재한다

newtype Product a =  Product { getProduct :: a }  
    deriving (Eq, Ord, Read, Show, Bounded)  
    
instance Num a => Monoid (Product a) where  
    mempty = Product 1  
    Product x `mappend` Product y = Product (x * y)  

In [81]:
import qualified Data.Monoid as Monoid 

Monoid.getProduct $ Monoid.Product 3 `mappend` Monoid.Product 9
Monoid.getProduct $ Monoid.Product 3 `mappend` Monoid.mempty
Monoid.getProduct $ Monoid.Product 3 `mappend` Monoid.Product 4 `mappend` Monoid.Product 2
Monoid.getProduct . Monoid.mconcat . map Monoid.Product $ [3,4,2]

Line 4: Monoid law, right identity
Found:
Monoid.Product 3 `mappend` Monoid.mempty
Why not:
Monoid.Product 3

27

3

24

In [83]:
import qualified Data.Monoid as Monoid 

Monoid.getSum $ Monoid.Sum 2 `mappend` Monoid.Sum 9  
Monoid.getSum $ mempty `mappend` Monoid.Sum 3  
Monoid.getSum . mconcat . map Monoid.Sum $ [1,2,3]  

Line 4: Monoid law, left identity
Found:
mempty `mappend` Monoid.Sum 3
Why not:
Monoid.Sum 3

11

3

6

## Bool도 모노이드

In [ ]:
-- OR 연산 혹은 AND 연산이 모노이드와 같다

newtype Any = Any { getAny :: Bool }  
    deriving (Eq, Ord, Read, Show, Bounded)  
    
instance Monoid Any where  
        mempty = Any False  
        Any x `mappend` Any y = Any (x || y)  

In [95]:
import qualified Data.Monoid as Monoid 

Monoid.getAny $ Monoid.Any True `Monoid.mappend` Monoid.Any False
Monoid.getAny $ Monoid.mempty `Monoid.mappend` Monoid.Any True
Monoid.getAny . Monoid.mconcat . map Monoid.Any $ [False, False, False, True]  
Monoid.getAny $ Monoid.mempty `Monoid.mappend` Monoid.mempty  

Line 4: Monoid law, left identity
Found:
Monoid.mempty `Monoid.mappend` Monoid.Any True
Why not:
Monoid.Any TrueLine 6: Monoid law, left identity
Found:
Monoid.mempty `Monoid.mappend` Monoid.mempty
Why not:
Monoid.memptyLine 6: Monoid law, right identity
Found:
Monoid.mempty `Monoid.mappend` Monoid.mempty
Why not:
Monoid.mempty

True

True

True

False

In [ ]:
newtype All = All { getAll :: Bool }  
        deriving (Eq, Ord, Read, Show, Bounded)  
        
instance Monoid All where  
    mempty = All True  
    All x `mappend` All y = All (x && y)  

In [98]:
Monoid.getAll $ Monoid.mempty `Monoid.mappend` Monoid.All True  
Monoid.getAll $ Monoid.mempty `Monoid.mappend` Monoid.All False  
Monoid.getAll . Monoid.mconcat . map Monoid.All $ [True, True, True]  
Monoid.getAll . Monoid.mconcat . map Monoid.All $ [True, True, False]  

True

False

True

False

## Ordering도 모노이드

In [101]:
1 `compare` 2
2 `compare` 2
3 `compare` 2

LT

EQ

GT

In [ ]:
instance Monoid Ordering where
    mempty = EQ  
    -- EQ가 어느 위치에 들어가더라도 결과는 같을 것
    LT `mappend` _ = LT  
    EQ `mappend` y = y  
    GT `mappend` _ = GT  
    -- 영어 단어를 비교하는 알고리즘과 같음
    -- 각 철자마다 비교하면서 같으면(EQ) 다음 계산을 하고
    -- 한 시점이라도 비교가 되면 그 값이 적용되고 뒷 값은 무시됨

In [103]:
LT `mappend` GT
GT `mappend` LT
mempty `mappend` LT
mempty `mappend` GT

Line 3: Monoid law, left identity
Found:
mempty `mappend` LT
Why not:
LTLine 4: Monoid law, left identity
Found:
mempty `mappend` GT
Why not:
GT

LT

GT

LT

GT

In [109]:
lengthCompare :: String -> String -> Ordering
-- 일반 코드
-- lengthCompare x y = let a = length x `compare` length y
--                         b = x `compare` y
--                     in if a == EQ then b else a
-- 모노이드 사용 코드
-- 좌항이 LT이면 뒷 값, 아니면 앞 값
lengthCompare x y = (length x `compare` length y) `mappend` (x `compare` y)

In [110]:
lengthCompare "zen" "ants"
lengthCompare "zen" "ant"

LT

GT

In [111]:
-- 모음의 숫자를 비교

lengthCompare :: String -> String -> Ordering
lengthCompare x y = (length x `compare` length y) `mappend`
                    (vowels x `compare` vowels y) `mappend`
                    (x `compare` y)
                        where vowels = length . filter (`elem` "aeiou")

In [112]:
lengthCompare "zen" "anna" -- 길이 비교
lengthCompare "zen" "ana" -- 모음 비교
lengthCompare "zen" "ann" -- 사전 비교

LT

LT

GT

In [113]:
-- Ordering 모노이드는 매우 많은 다른 조건들로 쉽게 비교할 수 있게 해주고
-- 가장 중요한 것부터 가장 덜 중요한 것까지 자신에게 넣을 수 있기 때문이다.

## Maybe도 모노이드

### 1. Maybe a 의 a도 모노이드

In [ ]:
instance Monoid a => Monoid (Maybe a) where  -- a가 모노이드 일때만 Maybe a도 모노이드
    mempty = Nothing  -- Nothing이 항등원
    Nothing `mappend` m = m  
    m `mappend` Nothing = m  
    Just m1 `mappend` Just m2 = Just (m1 `mappend` m2)  

In [114]:
Nothing `mappend` Just "andy"
Just LT `mappend` Nothing
Just (Sum 3) `mappend` Just (Sum 4)

-- 실패할수 있는 계산에 사용함으로써 실패한 값을 최대한 없애면서 진행할 수 있음

Just "andy"

Just LT

Just (Sum {getSum = 7})

### Maybe a의 a가 모노이드가 아닐때도 (First)

In [ ]:
newtype First a = First { getFirst :: Maybe a }  
    deriving (Eq, Ord, Read, Show)  

In [ ]:
instance Monoid (First a) where  
    mempty = First Nothing  
    -- Nothing이 항등원
    First (Just x) `mappend` _ = First (Just x)  
    First Nothing `mappend` x = x  
    -- 왼쪽 값을 그대로 주거나, 왼쪽값이 Nothing일때 오른쪽 값을 그대로 준다

In [116]:
Monoid.getFirst $ Monoid.First (Just 'a') `Monoid.mappend` Monoid.First (Just 'b')  
Monoid.getFirst $ Monoid.First Nothing `Monoid.mappend` Monoid.First (Just 'b')  
Monoid.getFirst $ Monoid.First (Just 'a') `Monoid.mappend` Monoid.First Nothing  

Just 'a'

Just 'b'

Just 'a'

In [117]:
Monoid.getFirst . Monoid.mconcat . map Monoid.First $ [Nothing, Just 9, Just 10]  
-- 많은 Maybe 값들이 있을때 어떤것이 Just 값인지 알고자할 때 유용

Just 9

### Maybe a의 a가 모노이드가 아닐때도 (Last)

위와 반대로 뒤의 값이 남는 Last도 있다

In [119]:
Monoid.getLast . Monoid.mconcat . map Monoid.Last $ [Nothing, Just 9, Just 10]  
Monoid.getLast $ Monoid.Last (Just "one") `Monoid.mappend` Last (Just "two")

Just 10

Just "two"

## 모노이드로 폴드하기

In [120]:
-- 리스트만이 폴드될 수 있는 유일한 데이터 구조는 아니다.
-- Foldable은 폴드 될 수 있는 것에 대한 데이터 클래스이다.
:info Foldable

type Foldable :: (* -> *) -> Constraint
class Foldable t where
  Data.Foldable.fold :: Monoid m => t m -> m
  foldMap :: Monoid m => (a -> m) -> t a -> m
  Data.Foldable.foldMap' :: Monoid m => (a -> m) -> t a -> m
  foldr :: (a -> b -> b) -> b -> t a -> b
  Data.Foldable.foldr' :: (a -> b -> b) -> b -> t a -> b
  foldl :: (b -> a -> b) -> b -> t a -> b
  Data.Foldable.foldl' :: (b -> a -> b) -> b -> t a -> b
  foldr1 :: (a -> a -> a) -> t a -> a
  foldl1 :: (a -> a -> a) -> t a -> a
  Data.Foldable.toList :: t a -> [a]
  null :: t a -> Bool
  length :: t a -> Int
  elem :: Eq a => a -> t a -> Bool
  maximum :: Ord a => t a -> a
  minimum :: Ord a => t a -> a
  sum :: Num a => t a -> a
  product :: Num a => t a -> a
  {-# MINIMAL foldMap | foldr #-}
  	-- Defined in ‘Data.Foldable’
instance Foldable (Applicative.Const m) -- Defined in ‘Data.Functor.Const’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Foldable Sum -- Defined in ‘Data.Foldable’
instance Foldable Product -- Defined in ‘Data.Foldable’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Foldable Last -- Defined in ‘Data.Foldable’
instance Foldable First -- Defined in ‘Data.Foldable’
instance Foldable (Either a) -- Defined in ‘Data.Foldable’
instance Foldable Dual -- Defined in ‘Data.Foldable’
instance Foldable f => Foldable (Ap f) -- Defined in ‘Data.Foldable’
instance Foldable f => Foldable (Alt f) -- Defined in ‘Data.Foldable’
instance Foldable ((,) a) -- Defined in ‘Data.Foldable’
instance Foldable Applicative.ZipList -- Defined in ‘Control.Applicative’

## Foldable

In [8]:
import qualified Data.Foldable as F

:info Foldable
-- type Foldable :: (* -> *) -> Constraint
-- class Foldable t where
-- Foldable t는 타입 인스턴스 인자 하나를 받는다
:t foldr
:t F.foldr

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

F.foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

type Foldable :: (* -> *) -> Constraint
class Foldable t where
  F.fold :: Monoid m => t m -> m
  foldMap :: Monoid m => (a -> m) -> t a -> m
  F.foldMap' :: Monoid m => (a -> m) -> t a -> m
  foldr :: (a -> b -> b) -> b -> t a -> b
  F.foldr' :: (a -> b -> b) -> b -> t a -> b
  foldl :: (b -> a -> b) -> b -> t a -> b
  F.foldl' :: (b -> a -> b) -> b -> t a -> b
  foldr1 :: (a -> a -> a) -> t a -> a
  foldl1 :: (a -> a -> a) -> t a -> a
  F.toList :: t a -> [a]
  null :: t a -> Bool
  length :: t a -> Int
  elem :: Eq a => a -> t a -> Bool
  maximum :: Ord a => t a -> a
  minimum :: Ord a => t a -> a
  sum :: Num a => t a -> a
  product :: Num a => t a -> a
  {-# MINIMAL foldMap | foldr #-}
  	-- Defined in ‘Data.Foldable’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Foldable (Either a) -- Defined in ‘Data.Foldable’
instance Foldable ((,) a) -- Defined in ‘Data.Foldable’

In [5]:
import qualified Data.Foldable as F

foldr (*) 1 [1,2,3]
F.foldr (*) 1 [1,2,3]

Line 3: Use product
Found:
foldr (*) 1
Why not:
product

6

6

In [6]:
import qualified Data.Foldable as F

F.foldl (+) 2 (Just 9)
F.foldr (||) False (Just True)

11

True

In [9]:
-- Foldable 인스턴스를 만드는 방법
-- 1) foldr을 직접 구현
-- 2) foldMap 함수를 구현
:t foldMap
-- foldMap :: (Monoid m, Foldable t) => (a -> m) -> t a -> m  
-- 1. (a->m): 폴드할 수 있는 구조 a 를 받아 모노이드 값을 반환하는 함수
-- 2. t a : 폴더블 값 안에 a
-- output : 모노이드 값

-- 1. 폴드할 수 있는 구조에 그 함수를 매핑하여 모노이드 값을 가지는 폴드할 수 있는 구조를 생성한다
-- 2.이들 모노이드 값들 간에 mappend 해서 하나의 모노이드 값으로 합친다

foldMap :: forall (t :: * -> *) m a. (Foldable t, Monoid m) => (a -> m) -> t a -> m

In [14]:
import qualified Data.Foldable as F

data Tree a = Empty | Node a (Tree a) (Tree a) deriving (Show, Read, Eq)  

instance F.Foldable Tree where  
    foldMap f Empty = mempty  
    foldMap f (Node x l r) = F.foldMap f l `mappend`  
                             f x           `mappend`  
                             F.foldMap f r  

In [15]:
testTree = Node 5  
            (Node 3  
                (Node 1 Empty Empty)  
                (Node 6 Empty Empty)  
            )  
            (Node 9  
                (Node 8 Empty Empty)  
                (Node 10 Empty Empty)  
            )  

In [22]:
F.foldl (+) 0 testTree 
:t F.foldl (+) 0 testTree 
-- F.foldMap (+) Node 3 ... `mappend` (+ 5) `mappend` F.foldMap (+) Node 9...
-- l tree
-- mempty `mappend` (+ 1) `mappend` mempty `mappend` (+ 3) `mappend` mempty `mappend` (+ 6) `mappend` mempty
-- (+ 1) `mappend` (+3) `mappend` (+6) ....
-- (+ 1) `mappend` (+3) `mappend` (+6)  `mappend` (+5)  `mappend` (+8)  `mappend` (+9)  `mappend` (+10)
-- => a는 Num, m은 a -> a 임
F.foldl (*) 1 testTree  

42

F.foldl (+) 0 testTree :: forall b. Num b => b

64800

In [26]:
import qualified Data.Foldable as F
import qualified Data.Monoid as Monoid

Monoid.getAny $ F.foldMap (\x -> Monoid.Any $ x == 3) testTree  
Monoid.getAny $ F.foldMap (\x -> Monoid.Any $ x > 15) testTree  

True

False

In [27]:
F.foldMap (\x -> [x]) testTree 
-- 리스트의 `mappend`는 ++ 이므로 아래와 같이 쭉 펼쳐진다

Line 1: Use :
Found:
\ x -> [x]
Why not:
(: [])

[1,3,6,5,8,9,10]